In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
df = pd.read_csv('data/COVID19MEXICO2025.csv')

In [4]:
df["ID_REGISTRO"].duplicated().sum()

0

In [5]:
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,INDIGENA,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_PCR,RESULTADO_PCR_COINFECCION,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL_COVID,CLASIFICACION_FINAL_FLU,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2025-03-04,0aa89f,1,10,9,2,15,9,3,1,2024-01-26,2024-01-25,9999-99-99,97,2,31,1,97,2,2,2,2,2,2,2,2,2,2,2,2,1,1,5,5,2,97,7,7,99,México,97,97
1,2025-03-04,g76bd10,1,4,32,2,32,32,56,2,2024-01-02,2024-01-02,9999-99-99,2,2,1,1,97,2,2,2,2,2,2,2,2,2,2,2,2,2,1,11,998,2,97,7,7,99,México,97,2
2,2025-03-04,gaa8ce3,1,12,9,1,9,15,13,1,2024-01-02,2024-01-01,9999-99-99,97,2,38,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,998,2,97,7,3,99,México,97,97
3,2025-03-04,g9637c9,1,4,25,1,25,25,12,2,2024-01-02,2024-01-01,9999-99-99,2,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,10,998,2,97,7,7,99,México,97,2
4,2025-03-04,g824984,1,4,32,2,32,32,17,2,2024-01-02,2024-01-02,9999-99-99,2,2,2,1,97,2,2,2,2,2,2,2,2,2,1,2,2,2,1,34,29,2,97,3,7,99,México,97,2


Se seleccionan unicamente los registros que sean confirmados y que sean casos positivos de COVID-19

Según el catalogo de datos, los valores 1,2,3 son casos confirmados 

In [6]:
df["CLASIFICACION_FINAL_COVID"].value_counts()

CLASIFICACION_FINAL_COVID
7    111695
6     84712
3     14648
5      4314
4       152
Name: count, dtype: int64

In [7]:
df = df[df["CLASIFICACION_FINAL_COVID"] == 3]

### Se crea una nueva variable que es saber si la persona falleció o no

In [8]:
df["DEFUNCION"] = np.where(df['FECHA_DEF'] == '9999-99-99', 2, 1)

### Se pasa a formato fecha

#### Primero se pasa a nulos los valores de 9999-99-99, que representan lo que no han fallecido

In [9]:
df['FECHA_DEF'] = df['FECHA_DEF'].replace('9999-99-99', np.nan)

In [10]:
df['FECHA_DEF'] = pd.to_datetime(df['FECHA_DEF'])

In [11]:
df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_ACTUALIZACION'] = pd.to_datetime(df['FECHA_ACTUALIZACION'])

Vamos a calcular el numero de dias que han pasado desde la fecha de ingreso/sintomas hasta la fecha de defuncion

In [12]:
df['DIAS_SINTOMAS_DEF'] = (df['FECHA_DEF'] - df['FECHA_SINTOMAS']).dt.days
df['DIAS_SINTOMAS_ACT'] = (df['FECHA_ACTUALIZACION'] - df['FECHA_SINTOMAS']).dt.days

In [13]:
df[df['DEFUNCION']== 1][['DIAS_SINTOMAS_DEF','DIAS_SINTOMAS_ACT']]

,DIAS_SINTOMAS_DEF,DIAS_SINTOMAS_ACT
446,16.0,428
767,9.0,428
2203,9.0,426
2445,0.0,421
2514,4.0,423
...,...,...
202943,20.0,24
210125,17.0,27
210627,3.0,10
210631,5.0,10


In [14]:
df["FECHA_ACTUALIZACION"].describe()

count                  14648
mean     2025-03-04 00:00:00
min      2025-03-04 00:00:00
25%      2025-03-04 00:00:00
50%      2025-03-04 00:00:00
75%      2025-03-04 00:00:00
max      2025-03-04 00:00:00
Name: FECHA_ACTUALIZACION, dtype: object

In [15]:
# df_2025 = df[df["FECHA_SINTOMAS"] > "2024-12-31"]

Seleccionamos las columnas que nos interesan para el análisis

In [16]:
df_cox = df[["DEFUNCION","DIAS_SINTOMAS_ACT","DIAS_SINTOMAS_DEF", "NEUMONIA", "DIABETES", "EPOC", "ASMA", "INMUSUPR", "HIPERTENSION", "CARDIOVASCULAR", "OBESIDAD", "RENAL_CRONICA", "TABAQUISMO","SEXO","EDAD"]]
# df_cox = df_2025[["DEFUNCION","DIAS_SINTOMAS_ACT","DIAS_SINTOMAS_DEF", "NEUMONIA", "DIABETES", "EPOC", "ASMA", "INMUSUPR", "HIPERTENSION", "OTRA_COM", "CARDIOVASCULAR", "OBESIDAD", "RENAL_CRONICA", "TABAQUISMO"]]

In [17]:
df_cox.head()

,DEFUNCION,DIAS_SINTOMAS_ACT,DIAS_SINTOMAS_DEF,NEUMONIA,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,SEXO,EDAD
4,2,427,NaN,2,2,2,2,2,2,2,1,2,2,2,2
6,2,428,NaN,2,2,2,2,2,2,2,2,2,2,1,37
17,2,427,NaN,2,2,2,2,2,2,2,2,2,2,1,45
20,2,427,NaN,2,2,2,2,2,2,2,2,2,2,1,27
23,2,428,NaN,2,2,2,2,2,2,2,2,2,2,2,33


In [18]:
df_cox["DEFUNCION"].value_counts()

DEFUNCION
2    13774
1      874
Name: count, dtype: int64

In [19]:
df_cox.shape

(14648, 15)

Camabiar valores para que 1 sea si (prensenta condicion o afirmacion de nombre de columna) y 0 sea no (no presenta condicion o negación de nombre de columna)

In [20]:
day_variables = df_cox[["DIAS_SINTOMAS_ACT","DIAS_SINTOMAS_DEF","EDAD"]]

In [21]:
df_cox = df_cox.drop(day_variables, axis=1)

In [22]:
df_cox.replace([1,2], [1,0], inplace=True)

Vamos a eliminar los registros con valores nulos

In [23]:
df_cox.replace([97,98,99], np.nan, inplace=True)

In [24]:
df_cox = df_cox.dropna()

In [70]:
df_cox["DEFUNCION"].value_counts(normalize=True)

DEFUNCION
0    0.940686
1    0.059314
Name: proportion, dtype: float64

In [26]:
df_cox.shape

(14617, 12)

In [27]:
df_cox = df_cox.astype(int)

In [28]:
df_cox = df_cox.join(day_variables, how='left')

In [29]:
df_cox.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14617 entries, 4 to 214657
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DEFUNCION          14617 non-null  int32  
 1   NEUMONIA           14617 non-null  int32  
 2   DIABETES           14617 non-null  int32  
 3   EPOC               14617 non-null  int32  
 4   ASMA               14617 non-null  int32  
 5   INMUSUPR           14617 non-null  int32  
 6   HIPERTENSION       14617 non-null  int32  
 7   CARDIOVASCULAR     14617 non-null  int32  
 8   OBESIDAD           14617 non-null  int32  
 9   RENAL_CRONICA      14617 non-null  int32  
 10  TABAQUISMO         14617 non-null  int32  
 11  SEXO               14617 non-null  int32  
 12  DIAS_SINTOMAS_ACT  14617 non-null  int64  
 13  DIAS_SINTOMAS_DEF  867 non-null    float64
 14  EDAD               14617 non-null  int64  
dtypes: float64(1), int32(12), int64(2)
memory usage: 1.6 MB


In [30]:
df_cox.columns

Index(['DEFUNCION', 'NEUMONIA', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA',
       'TABAQUISMO', 'SEXO', 'DIAS_SINTOMAS_ACT', 'DIAS_SINTOMAS_DEF', 'EDAD'],
      dtype='object')

En los dias transcurridos hasta el evento, para los pacientes que fallecieron, los valores se reemplazan por los días desde la fecha del inicio de los sintomas, hasta la fecha de defuncion. Para los pacientes que no fallecieron, el valor de dias transcurridos se reemplazan por los dias desde el inicio de los sintomas hasta la ultima fecha de actualización del conjunto de datos(representa el ultimo dia de estudio)

In [31]:
df_cox["DIAS_TRANSCURRIDOS"] = np.where(df_cox['DEFUNCION'] == 0, df_cox["DIAS_SINTOMAS_ACT"], df_cox["DIAS_SINTOMAS_DEF"])

In [32]:
df_cox

,DEFUNCION,NEUMONIA,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,SEXO,DIAS_SINTOMAS_ACT,DIAS_SINTOMAS_DEF,EDAD,DIAS_TRANSCURRIDOS
4,0,0,0,0,0,0,0,0,1,0,0,0,427,NaN,2,427.0
6,0,0,0,0,0,0,0,0,0,0,0,1,428,NaN,37,428.0
17,0,0,0,0,0,0,0,0,0,0,0,1,427,NaN,45,427.0
20,0,0,0,0,0,0,0,0,0,0,0,1,427,NaN,27,427.0
23,0,0,0,0,0,0,0,0,0,0,0,0,428,NaN,33,428.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214130,0,0,0,0,0,0,0,0,0,0,0,1,7,NaN,47,7.0
214182,0,0,0,0,0,1,0,1,0,0,1,1,9,NaN,42,9.0
214559,0,0,1,0,0,0,1,0,0,0,0,1,10,NaN,69,10.0
214561,0,1,0,0,0,1,0,0,0,0,0,1,7,NaN,54,7.0


In [33]:
df_cox.drop(["DIAS_SINTOMAS_ACT", "DIAS_SINTOMAS_DEF"], axis=1, inplace=True)

In [34]:
df_cox.columns

Index(['DEFUNCION', 'NEUMONIA', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA',
       'TABAQUISMO', 'SEXO', 'EDAD', 'DIAS_TRANSCURRIDOS'],
      dtype='object')

In [36]:
df_cox.columns = ['DEFUNCION', 'NEUMONIA', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'SEXO','EDAD', 'DIAS_SINTOMAS_DEF']

In [38]:
df_cox.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14617 entries, 4 to 214657
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DEFUNCION          14617 non-null  int32  
 1   NEUMONIA           14617 non-null  int32  
 2   DIABETES           14617 non-null  int32  
 3   EPOC               14617 non-null  int32  
 4   ASMA               14617 non-null  int32  
 5   INMUSUPR           14617 non-null  int32  
 6   HIPERTENSION       14617 non-null  int32  
 7   CARDIOVASCULAR     14617 non-null  int32  
 8   OBESIDAD           14617 non-null  int32  
 9   RENAL_CRONICA      14617 non-null  int32  
 10  TABAQUISMO         14617 non-null  int32  
 11  SEXO               14617 non-null  int32  
 12  EDAD               14617 non-null  int64  
 13  DIAS_SINTOMAS_DEF  14617 non-null  float64
dtypes: float64(1), int32(12), int64(1)
memory usage: 1.5 MB


In [39]:
df_cox = df_cox.astype(int)

# Descripción de las caracteristicas

In [68]:
df_cox[df_cox["DEFUNCION"] == 0].EDAD.describe()

count    13750.000000
mean        42.690545
std         21.902382
min          0.000000
25%         28.000000
50%         41.000000
75%         58.000000
max        109.000000
Name: EDAD, dtype: float64

In [69]:
df_cox[df_cox["DEFUNCION"] == 1].EDAD.describe()

count    867.000000
mean      65.346021
std       22.334177
min        0.000000
25%       56.000000
50%       71.000000
75%       81.000000
max      102.000000
Name: EDAD, dtype: float64

In [85]:
df_cox[df_cox["SEXO"] == 0].DEFUNCION.value_counts(normalize=True)

DEFUNCION
0    0.91178
1    0.08822
Name: proportion, dtype: float64

In [86]:
df_cox[df_cox["SEXO"] == 1].DEFUNCION.value_counts(normalize=True)

DEFUNCION
0    0.960345
1    0.039655
Name: proportion, dtype: float64

In [43]:
df_cox[df_cox["DEFUNCION"] == 0].DIAS_SINTOMAS_DEF.describe()

count    13750.000000
mean       285.232145
std         99.418546
min          4.000000
25%        218.000000
50%        265.000000
75%        382.000000
max        428.000000
Name: DIAS_SINTOMAS_DEF, dtype: float64

In [44]:
df_cox[df_cox["DEFUNCION"] == 1].DIAS_SINTOMAS_DEF.describe()

count    867.000000
mean      11.189158
std       16.279990
min        0.000000
25%        4.000000
50%        8.000000
75%       14.000000
max      373.000000
Name: DIAS_SINTOMAS_DEF, dtype: float64

In [50]:
df_cox[df_cox["DEFUNCION"] == 1].DIAS_SINTOMAS_DEF.describe()

count    867.000000
mean      11.189158
std       16.279990
min        0.000000
25%        4.000000
50%        8.000000
75%       14.000000
max      373.000000
Name: DIAS_SINTOMAS_DEF, dtype: float64

In [ ]:
comorbilidades = [['NEUMONIA', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA',
       'TABAQUISMO']]

In [79]:
for comorbilidad in comorbilidades:
    print("")
    print("COMORBILIDAD: " + comorbilidad)
    print(df_cox[df_cox[comorbilidad] == 1].DEFUNCION.value_counts(normalize=True))


COMORBILIDAD: NEUMONIA
DEFUNCION
0    0.781368
1    0.218632
Name: proportion, dtype: float64

COMORBILIDAD: DIABETES
DEFUNCION
0    0.850163
1    0.149837
Name: proportion, dtype: float64

COMORBILIDAD: EPOC
DEFUNCION
0    0.807606
1    0.192394
Name: proportion, dtype: float64

COMORBILIDAD: ASMA
DEFUNCION
0    0.957404
1    0.042596
Name: proportion, dtype: float64

COMORBILIDAD: INMUSUPR
DEFUNCION
0    0.891192
1    0.108808
Name: proportion, dtype: float64

COMORBILIDAD: HIPERTENSION
DEFUNCION
0    0.861604
1    0.138396
Name: proportion, dtype: float64

COMORBILIDAD: CARDIOVASCULAR
DEFUNCION
0    0.82397
1    0.17603
Name: proportion, dtype: float64

COMORBILIDAD: OBESIDAD
DEFUNCION
0    0.926108
1    0.073892
Name: proportion, dtype: float64

COMORBILIDAD: RENAL_CRONICA
DEFUNCION
0    0.808547
1    0.191453
Name: proportion, dtype: float64

COMORBILIDAD: TABAQUISMO
DEFUNCION
0    0.865103
1    0.134897
Name: proportion, dtype: float64
